---

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-10-04 07:27:09--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.1’

SO_vectors_200.bin? 100%[===================>]   1.35G   511KB/s    in 47m 35s 

2023-10-04 08:14:46 (497 KB/s) - ‘SO_vectors_200.bin?download=1.1’ saved [1453905423/1453905423]



In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/drive')
bin_file_path = '/content/SO_vectors_200.bin?download=1.1'

drive_path = '/content/drive/MyDrive/'

# Копируем бин файл на Google Диск
shutil.copy(bin_file_path, drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/SO_vectors_200.bin?download=1.1'

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/SO_vectors_200.bin?download=1.1'
colab_path = '/content'
#Копируем бин файл в колаб
!cp $drive_path $colab_path

Mounted at /content/drive


In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1.1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

In [ ]:
wv_embeddings.most_similar("dog")

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
# method most_similar
wv_embeddings.most_similar("dog", topn=5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

In [ ]:
wv_embeddings.similarity("dog", "cat")

0.6852341

In [ ]:
# Загрузка google embedding
from google.colab import drive

drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
colab_path = '/content'
#Копируем бин файл в колаб
!cp $drive_path $colab_path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Воспользуемся word2vec-google-news-300 для дополнительного анализа

google_news_embeddings = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
word = 'dog'
if word in google_news_embeddings:
    print(google_news_embeddings[word].dtype, google_news_embeddings[word].shape)

float32 (300,)


Формат stack overflow и google embadding не отличается, вектор стал длиннее



In [ ]:
# Топ 5 близких слов к слову "dog" в GNE
google_news_embeddings.most_similar("dog", topn=5)

[('dogs', 0.8680489659309387),
 ('puppy', 0.8106428384780884),
 ('pit_bull', 0.780396044254303),
 ('pooch', 0.7627376914024353),
 ('cat', 0.7609457969665527)]

In [ ]:
google_news_embeddings.similarity("dog", "cat")

0.76094574

Если рассматривать google news embeddings, то cat входит в топ 5 схожих слов с dog. При этом коэффициент схожести выше, чем у stack overflow; если рассматривать топ 5 слов, то есть только одно совпадение (и оно логичное) dogs

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    words = tokenizer.tokenize(question)
    vectors = [embeddings[word] for word in words if word in embeddings]
    if vectors:
      return np.mean(vectors, axis=0)
    if not vectors:
      return np.zeros(dim)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
# В stack overflow
round(question_to_vec("I love neural networks", wv_embeddings, tokenizer)[2], 2)

-1.29

In [ ]:
# В google news
round(question_to_vec("I love neural networks", google_news_embeddings, tokenizer)[2], 2)

0.02

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?


Для поиска максимума в разности нужно, чтобы уменьшаемое было максимальным, а вычитаемое минимальным
1.   Hist@47 - доля правильных предсказаний в первых 47 предсказаниях, ее максимум = 1
2.   DCG@1 - показывает насколько сильно первое предсказание соответствует поисковому запросу (аналогия с поиском для понимания), ее минимум = 0, например, гуглишь фотографию кота, а тебе выдает подсолнух.

Вывод: Максимальная разница равна 1.



<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
def DCG_at_K(rank_qi, k):
  return 1/np.log2(1+rank_qi) * (rank_qi <= k)

In [ ]:
DCG_at_K(9, 10)

0.3010299956639812

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = np.array([rank_qi <= k for rank_qi in dup_ranks])
    return hits_value.mean()

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = np.array([1/np.log2(1+rank_qi) * (rank_qi <= k) for rank_qi in dup_ranks])
    return dcg_value.mean()

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k

correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    words = tokenizer.tokenize(question)
    vectors = [embeddings[word] for word in words if word in embeddings]
    if vectors:
      return np.mean(vectors, axis=0)
    if not vectors:
      return np.zeros(dim)

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vector = question_to_vec(question,embeddings, tokenizer, dim).reshape(1, -1)
    candidates_vectors = []
    for i, candidate in enumerate(candidates):
      candidate_vector = question_to_vec(candidate, embeddings, tokenizer, dim).reshape(1, -1)
      candidates_vectors.append((i, candidate, cosine_similarity(question_vector,
                                                                 candidate_vector)))

    candidates_vectors.sort(key=lambda x: x[2], reverse=True)

    # Возвращение отсортированного списка пар (позиция, кандидат)
    return [(position, candidate) for position, candidate, _ in candidates_vectors]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(*ranks, sep='\n')
        print()

(1, 'C# create cookie from string and send it')
(0, 'Convert Google results object (pure js) to Python object')
(2, 'How to use jQuery AJAX for an outside domain?')

(1, 'WPF- How to update the changes in list item of a list')
(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')



In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, google_news_embeddings, tokenizer)
        print(*ranks, sep='\n')
        print()

(1, 'C# create cookie from string and send it')
(0, 'Convert Google results object (pure js) to Python object')
(2, 'How to use jQuery AJAX for an outside domain?')

(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')
(1, 'WPF- How to update the changes in list item of a list')



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?



*   Для stack overflow embedding 102
*   Для google news embedding 021



Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/stackoverflow_similar_questions.zip'
colab_path = '/content'
#Копируем бин файл в колаб
!cp $drive_path $colab_path

Mounted at /content/drive


In [ ]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
     data = []
     with open(filename, 'r', encoding='utf-8') as file:
      for line in file:
        data.append(line.strip())
      return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
     print(i + 1, len(validation_data[i]), validation_data[i])

1 54516 How to print a binary heap tree without recursion?	How do you best convert a recursive function to an iterative one?	How can i use ng-model with directive in angular js	flash: drawing and erasing	toggle react component using hide show classname	Use a usercontrol from another project to current webpage	~ Paths resolved differently after upgrading to ASP.NET 4	Materialize datepicker - Rendering when an icon is clicked	Creating PyPi package - Could not find a version that satisfies the requirement iso8601	How can I analyze a confusion matrix?	How do I declare a C array in Swift?	Using rand() when flipping a coin and rolling a die	Handling a JSON field with a special character in its name in Java	React Native select row on ListView when push it	Get 'creation_time' of video using ffmpeg and regex	Does row exist and multiple where	How to specify a classifier in a gradle dependency's dependency?	Using $unwind on multiple documents	Visual Studio Code: Missing X509Certificate2UI	Develop

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 3760
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line.split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.288 | Hits@   1: 0.288
DCG@   5: 0.348 | Hits@   5: 0.402
DCG@  10: 0.368 | Hits@  10: 0.465
DCG@ 100: 0.413 | Hits@ 100: 0.691
DCG@ 500: 0.437 | Hits@ 500: 0.876
DCG@1000: 0.450 | Hits@1000: 1.000


Выводы: чем выше К, тем лучше метрики (это логично, так как при большем К будет больше значений для включения правельного ответа, то же самое и с качеством ответов, если взять больше вопросов, то и вероятность того, что нам попадется нужный - выше)

### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
words = [question.split('\t') for question in train_data]

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,            # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=5,      # consider words that occured at least 5 times
                              window=5).wv      # оптимальное значение окна - это 5 слов (обычно оно берется в значении от 2 до 10)

In [ ]:
wv_ranking = []
max_validation_examples = 3760
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line.split('\t')
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 1.000 | Hits@   1: 1.000
DCG@   5: 1.000 | Hits@   5: 1.000
DCG@  10: 1.000 | Hits@  10: 1.000
DCG@ 100: 1.000 | Hits@ 100: 1.000
DCG@ 500: 1.000 | Hits@ 500: 1.000
DCG@1000: 1.000 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:




*   N-grams, так как суть заключается в кодировке словосочетаний, следовательно сохраняется контекст, следовательно и возможностей для правильного ответа больше, например, можно создать хороший эмбеддинг, а потом с помощью n_gramms обучить модель со словосочетаниями-синонимами (достаточно найти максимальное косинусное расстояние) и получится пушка-бомба
*   Когда как, с одной стороны, она сокращает размерность пространства слов, устранив различные формы одного и того же слова, с другой, мы теряем информацию, особенно если различные формы слов имеют разные значения или контексты.
*   Второй. Причина, мы созлали его на уже заданной тематике, если же рассматривать предобученные, то они не будут так хорошо работать с нашими данными, но будут себя показывать гораздо лучше в общих задачах
*   Во втором случае качество 1, в первом, так как загруженные эмбеддинги созданы для определения общих случаев, а не отдельных тематик.
*   Задействовать более сложные алгоритмы для нахождения зависимостей между словами, использовать предобученные модели для поиска схожести предложений и т. д.



